In [ ]:
from datetime import datetime, date, timedelta
from dateutil.parser import parse
import math
import os
import sys
import time

import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as colors
import matplotlib.image as mpimg
from matplotlib.offsetbox import AnchoredText
from matplotlib.backends.backend_pdf import PdfPages

import mapclassify as mc

from pathlib import Path
import requests,json
from tabulate import tabulate


def read_json_ESRI():
    #
    # Connect to RKI ESRI data service to get the Landkreis geodata
    # and the population data of the Landkreise
    # 
    # Input : None
    # Output: result: GeoDataFrame of the RKI Landkreise
    #
    url="https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?"

    parameter = {
        'referer':'https://www.mywebapp.com',
        'user-agent':'python-requests/2.9.1',
        'where': '1=1', # Welche landkreise sollen zurück gegeben werden
        'outFields': 'RS,GEN,county,EWZ,BL_ID,BL,EWZ_BL', # Rückgabe aller Felder
        'returnGeometry': True, # Mit Geometrien
        'f':'geojson', # Rückgabeformat, hier JSON
        'cacheHint': True # Zugriff über CDN anfragen
    }
    
    result = requests.get(url=url, params=parameter) #Anfrage absetzen
    resultjson = json.loads(result.text) # Das Ergebnis JSON als Python Dictionary laden
    result=gpd.GeoDataFrame.from_features(resultjson)
    result.set_crs(epsg=4326, inplace=True)
    return result



def read_json_BL():
    #
    # Connect to RKI ESRI data service to get the Bundeslaender geodata
    # 
    # Input : None
    # Output: result: GeoDataFrame of the RKI Bundeslaender
    #
    
    url="https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?"
    
    parameter = {
        'referer':'https://www.mywebapp.com',
        'user-agent':'python-requests/2.9.1',
        'where': '1=1', # Welche landkreise sollen zurück gegeben werden
        'outFields': 'LAN_ew_GEN,LAN_ew_BEZ', # Rückgabe aller Felder
        'returnGeometry': True, # Mit Geometrien
        'f':'geojson', # Rückgabeformat, hier JSON
        'cacheHint': True # Zugriff über CDN anfragen
    }
    result = requests.get(url=url, params=parameter) #Anfrage absetzen
    resultjson = json.loads(result.text) # Das Ergebnis JSON als Python Dictionary laden
    result=gpd.GeoDataFrame.from_features(resultjson)
    result.set_crs(epsg=4326, inplace=True)
    return result



def get_covid_data_from_RKI(outfile):
    #
    # Connect to RKI data service to download the latest covid database (full version)
    # 
    # Input : outfile, name of csv-file which has the complete covid data 
    # Output: filesize of the csv-file
    #
    covid_url = \
'https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data' 
    r = requests.get(covid_url, allow_redirects=True)
    filesize = open(outfile, 'wb').write(r.content)
    return(filesize)



def read_covid_database(covid_file):
    #
    # Wrapper function for downloading and reading the covid database
    # 
    # Input : covid-file, name of csv-file which has the complete covid data 
    # Output: covid, the complete covid dataframe
    #
    # Remark: Since downloading takes some time, it ist tested if an actual 
    # version of the csv-file already exists on disc
    #
    covid_path=Path(covid_file)
    if not covid_path.is_file():
        print('Loading covid data to: ', covid_file)
        get_covid_data_from_RKI(covid_file)
    else:
        print('Using already existing file ', covid_file)
    covid = pd.read_csv(covid_file, parse_dates=['Meldedatum'])
    return(covid)



def get_bundeslad_geodata(bula_file):
    #
    # Read Bundesland geodaten
    #
    # Input: bula_file, Datenfile
    # Output: bland, GeoDataFrame of Bundeslaender
    #
    bula_path=Path(bula_file)
    if not bula_path.is_file():
        bland = read_json_BL()
        bland.to_file(bula_file)
    else:
        bland = gpd.read_file(bula_file)
    return bland



def get_kreise_geodata(kreise_file):
    #
    # Read Landkreis geodaten
    #
    # Input: kreise_file, Datenfile
    # Output: kreise, GeoDataFrame of Landkreise
    #
    kreise_path=Path(kreise_file)
    if not kreise_path.is_file():
        kreise = read_json_ESRI()
        kreise.to_file(kreise_file)
    else:
        kreise = gpd.read_file(kreise_file)
    return(kreise)



def extract_einwohnerzahlen(kreise):
    #
    # Returns table of populations from Landkreise, 
    # Bundesländer und Germany
    # 
    # Input : kreise, ESRI Landkreis Geodataframe
    # Output: einwohner, Datafram with all the population 
    # data (Landkreise, bundeslaendern, germany) 
    
    ewz=pd.DataFrame(kreise)

    bl=ewz.groupby('BL').EWZ_BL.max()
    lk=ewz.groupby('county').EWZ.max()

    summe_ew_deutschland = bl.sum()
    
    einwohner=pd.concat([lk,bl])
    einwohner.loc['Deutschland']=summe_ew_deutschland

    return(einwohner)



def get_tagesinzidenzen(covid, start_date, end_date):
    # 
    # This s one of the main routines:
    # Generate a datelist for every day of the pandemic and 
    # use this list as an index to create an empty dataframe
    #
    # Input: covid, Covid dataframe from RKI
    # Input: start, end: Starting and end-date for the resulting dataframe
    # Output: result, the complete Inzidenz table for all Kreise,
    #         Bundeslaender and Germany for every day from start to end
    #
    tagesliste = pd.date_range(start_date,end_date)
    result=pd.DataFrame(index=tagesliste)
    
    filt= (covid.NeuerFall!=-1)
    covid2=covid[filt]
    
    # Main grouping steps for Landkreise, Bundesländer und Deutschland gesamt
    Faelle_lk = covid2.groupby(['Landkreis', 'Meldedatum']).AnzahlFall.sum()
    Faelle_bl = covid2.groupby(['Bundesland', 'Meldedatum']).AnzahlFall.sum()
    Faelle_d  = covid2.groupby('Meldedatum').AnzahlFall.sum()

    lk_auswahl=landkreise
    for i in lk_auswahl:
        tmp=Faelle_lk[i]
        result = pd.concat([result, tmp.rename(i)], axis=1)

    bl_auswahl=bundeslaender
    for i in bl_auswahl:
        tmp=Faelle_bl[i]
        result = pd.concat([result, tmp.rename(i)], axis=1)
    
    result = pd.concat([result,Faelle_d.rename('Deutschland')], axis=1)
    
    return(result)


def new_infections(ort):
    
    # Get Series of new infections
    #
    # input ort: list of Bundesland, landkreis or 'Deutschland'
    # output: Series with new infections
    #
    
    neu = pd.Series(index=ort, dtype=int)

    filter_n=filter1=(covid.NeuerFall==-1) | (covid.NeuerFall==1)
    new_covid = covid[filter_n]
    
    new_covid_d  = pd.Series(covid[filter_n].AnzahlFall.sum(), index=['Deutschland'])
    new_covid_bl = new_covid.groupby('Bundesland').AnzahlFall.sum()
    new_covid_lk = new_covid.groupby('Landkreis').AnzahlFall.sum()
    new_covid_list = [new_covid_lk , new_covid_bl , new_covid_d]
    neu.update(pd.concat(new_covid_list))
    
    return(neu)

def covid_db_stats():
    #
    # print out some infos about the covid database
    #
    if check_panbegin:
        panbegin = 'Startdatum in covid DB ist ok.'
    else:
        panbegin = 'Unplausibilität in Startdatum der Pandemie in den covid Daten'
    if check_meta:
        meta = 'Landkreis-, Bundesland- und Einwohnerzahl sind ok!'
    else:
         meta = 'Unplausibilität in Landkreis-, Bundesland- oder Einwohnerzahl!'
    return '''
    {0}
    {1}
    
    Start Erfassung      : {2} 
    Ende  Erfassung      : {3}
    Einwohner Deutschland: {4}
    Anzahl Bundesländer  : {5}
    Anzahl Landkreise    : {6}
    Summe Neuinfektionen : {7}
    Anzahl Landkreise mit 0 Neuinfektionen   : {8}
    Anzahl Landkreise mit < 10 Neuinfektionen: {9}
    '''.format(
        panbegin,
        meta,   
        start.strftime('%d.%m.%Y'),
        ende.strftime('%d.%m.%Y'), 
        einwohner['Deutschland'],
        len(bundeslaender),
        len(landkreise),
        neuinfektionen['Deutschland'],
        len(neuinfektionen[neuinfektionen == 0 ]),
        len(neuinfektionen[neuinfektionen < 10 ])
    )

#
#
# Graphic section
#
#

def didt_hist(data, 
              x='Prozentuale Änderung der 7 Tage Inzidenz/100T EW',
              y='Antzahl Land-/Stadtkreise',
              t='Anz. Landkreise über Inzidenzändeung in %',
              ref='gegenüber vor x Tagen',
              bins=list(range(-100,110,10)),
              savefig = False,
              file='Histogram'
             ):
    #
    # Histogramm Wrapper to show a histogramm for 
    # number of landkreise with a specific change in Inzidenz
    #
    fig, ax = plt.subplots(1,1,figsize=(10,6))
    data.hist(ax=ax, bins=bins)
              
    ax.set_ylabel(y)
    ax.set_xlabel(x)
    ax.set_title(t+ref)
    ax.set_xlim(bins[0],bins[-1])
    ax.text(0.01,0.02,copyright2, rotation=90, transform=ax.transAxes)

    neg=len(data[data<0])
    pos=len(data[data>=0])
    mean=data.mean()
           
    Summary7 = 'Stand '+ ende.strftime('%d.%m.%Y') \
+ '\n' + str(neg) + ' Landkreise fallend ' \
+ '\n' + str(pos) + ' Landkreise steigend ' \
+ '\nMittlere Änderung ' + '{:.2f}'.format(mean) + '%'

    at = AnchoredText(
        Summary7, prop=dict(size=11), frameon=True, loc='upper right')
    at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
    ax.add_artist(at)

    if savefig:
        histplotfile=ende.strftime('%y%m%d') + '_' + file + '.png'
        fig.savefig(histplotfile, dpi=300)
    return(fig)


def print_tabular_overview(tdata, 
                           title='Kreise Top 5 Änderungen',
                           savefig=False,
                           file='Table_Overview'
                          ):
    #
    # Table Wrapper to show a table
    # of landkreise with a specific change in Inzidenz
    #
    fig, ax = plt.subplots(1,1, figsize=(10,2))
    
    top=tdata.reset_index()
    top=top.round(2)

    table = ax.table(cellText=top.values, 
                     colLabels=["Kreis","Inzidenz","Änd. Vortag [%]", 
                                "Änd. Vorwoche [%]", "Fälle"], loc='center')
    table.scale(1,1.2)

    ax.set_title(title, fontsize=12)
    ax.axis('off')
    if savefig:
        tableplotfile=ende.strftime('%y%m%d') + '_' + file + '.png'
        fig.savefig(tableplotfile, dpi=300)
    return(fig)



def inzidenz_lineplot(overview,topscorer,filename='Inzidenzen',savefig=False):
    #
    # Linplot wrapper to show a lineplot
    # of several landkreise
    # Input: overview: Landkreise/Bundeslaender plotted with thick lines
    # input: topscorer: Landkreise/Bundeslaender plotted with thin lines
    #
    fig, ax = plt.subplots(1,1, figsize=(10,6))
    
    ax.grid()
    ax.set_title('7 Tage Neuinfektionen / 100.000 EW',fontsize=16)
    ax.set_xlabel('Datum', fontsize=12)
    ax.set_ylabel('Inzidenz', fontsize=12)


    for i in overview:
        if i == 'Deutschland':
            lw=3
        else:
            lw = 2
        ax.plot(result_s7[i], label=i, linewidth=lw)
    
    for i in topscorer:
        lw = 1
        ax.plot(result_s7[i], label=i, linewidth=lw)
    
    ax.set_xlim([datetime(2021,10,15),datetime(2022,2,15)])
    ax.set_yscale('log')

    ax.set_ylim([10,2000])
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    ax.legend(loc='best', fontsize=12)
    ax.text(0.02,.96,copyright2, transform=ax.transAxes)

    txt='Stand '+ ende.strftime('%d.%m.%Y')
    ax.text(.8,.96,txt, transform=ax.transAxes)

    if savefig:
        inzplotfile=ende.strftime('%y%m%d') + '_' + filename + '.png'
        fig.savefig(inzplotfile, dpi=100)
    return(fig)


    
def geoplot(data, color='INZ', cmap='GnBu', bins=[5, 20, 35, 50, 100, 200, 300, 1000],
            title='Inzidenzkarte_Deutschland',
            filename='Inzidenzkarte_Deutschland',
            savefig=False
           ):
    #
    # Mainplot mit Bundeslandgrenzen
    #
    # input: Geodataframe of Landkreise with additional columns containing
    #        Inzidenz Numbers od Delta-Inzidenz Numbers
    #
    title   = title + ' am ' + ende.strftime('%d.%m.%Y')

    fig, ax = plt.subplots(1,1, figsize=(10,12), dpi=100)

    #ax = tmpkreise.plot(figsize=(15,15), edgecolor='grey', column='INZ', cmap='tab10', scheme='MaximumBreaks', legend=True)
    #ax = tmpkreise.plot(figsize=(15,15), edgecolor='grey', column='INZ', cmap='tab10', scheme='Quantiles', legend=True)

    data.plot(ax=ax, edgecolor='grey', column=color, cmap=cmap, 
              scheme='UserDefined', classification_kwds={'bins': bins},
              legend=True, legend_kwds={'loc': 'upper left', 'fontsize': '8'})

    ax2=bland.plot(ax=ax, edgecolor='white', color='none')

    ax.axis('off')
    ax.set_title(title)
    ax.text(5.5,47,copyright2)

    if savefig:
        diagram_file = ende.strftime('%Y%m%d') + "_" + filename + '.png'
        plt.savefig(diagram_file, dpi=300)

    return(fig)



def inz_overview(overview):
    #
    # print an Inzidenz overview
    #
    inztop5_today = dinz_dt.sort_values(by='INZ').iloc[-5:]
    top3_dinz_dt7 = dinz_dt.sort_values(by='DINZ7').iloc[-3:]


    head1 = 'Überblick am: {0}\n\n'.format(ende.strftime('%d.%m.%Y'))

    table=[]
    for i in overview:
        a=[round(result_s7.iloc[-1][i],1),neuinfektionen[i],i]
        table.append(a)

    data1=tabulate(table, headers=['Inzidenz', 'Neue Fälle', 'Kreis'])
    
    
    head2 = '\n\nTop Änderungen gegenüber Vorwoche:\n\n'

    table=[]
    for i in list(top3_dinz_dt7.index):
        a=[round(result_s7.iloc[-1][i],1), 
           neuinfektionen[i],
           round(top3_dinz_dt7.loc[i,'DINZ7'],0),i
           ]
        table.append(a)

    data2 = tabulate(table, headers=['Inzidenz', 'Neue Fälle', 'Änderung [%]', 'Kreis'])


    head3 = '\n\nTop Inzidenzen:\n\n'

    table=[]
    for i in list(inztop5_today.index):
        a=[round(result_s7.iloc[-1][i],1),neuinfektionen[i],i]
        table.append(a)

    data3 = tabulate(table, headers=['Inzidenz', 'Neue Fälle', 'Kreis'])
    
    
    return head1+data1+head2+data2+head3+data3 


def plot_landesinfo(land, savefig = False, filename='land_info'):
    #
    # print an overview for a Bundesland
    # Input: land: Bundesland to plot
    #
    fig = plt.figure(figsize=(10,6))

    gs = fig.add_gridspec(2,2, width_ratios=[3, 4], height_ratios=[2,3])
    
    ax1 = fig.add_subplot(gs[:, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 1])

    filter = tmpkreise.BL == land
    tmp=tmpkreise.loc[filter]
    bins=[25, 50, 100, 200, 500, 1000, 2000]

    inzidenz=result_s7.iloc[-1][land]
    faelle=neuinfektionen[land]
    faelle=str(int(faelle))
    faelle_txt = faelle + ' Neuinfektionen'

    sum_faelle = str(int(result[land].sum()))
    sum_faelle_txt = sum_faelle + ' Fälle seit Beginn'

    prefix=''
    wochendelta = int(dinz_dt.loc[land,'DINZ7'])
    if wochendelta > 0:
        prefix='+'
    wochendelta=str(wochendelta)
    wochendelta_txt = 'Wochentrend: ' + prefix + wochendelta + '%'
    
    ax1.axis(False)
    # ax1.set_title(land)
    tmp.plot(ax=ax1, column='INZ', cmap='GnBu',
            scheme='UserDefined', classification_kwds={'bins': bins},
            legend=True, 
            legend_kwds={'bbox_to_anchor': (.2, 0.2), 'fontsize': '8'})
    
    leg = ax1.get_legend()
    i=0
    for lab in bins: 
        leg.get_texts()[i].set_text(lab)
        i+=1
    
    minx, miny, maxx, maxy = tmp.total_bounds
    ax1.set_xlim(minx, maxx)
    ax1.set_ylim(miny, maxy)
    
    
    ax2.axis(False)
    ax2.text(-1, .9,'Situation in ' + land, size=16)
    
    ax2.text(0, .7,'7-Tage Inzidenz:', size=12)
    ax2.text(0, .5, round(inzidenz,1), size=24)
    ax2.text(0, .3, faelle_txt, size=12)
    ax2.text(.5, .5, wochendelta_txt, size=12)
    ax2.text(.5, .3, sum_faelle_txt, size=12)

    ax3.set_title('Verlauf der täglichen 7-Tage Inzidenz')
    ax3.grid(True)
    ax3.plot(result_s7[land], label=land, linewidth=2)
    ax3.fill_between(result_s7[land].index, result_s7[land].values, color='lightgrey')
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%m/%y'))
    for label in ax3.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
         
    if savefig:
        diagram_file = ende.strftime('%Y%m%d') + "_" + filename + '_' + land + '.png'
        plt.savefig(diagram_file, dpi=300)
    
    return(fig)


def text_via_figure(text):
    #
    # text output via figure
    #
    fig, ax = plt.subplots(1,1,figsize=(12,6))
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    ax.set_axis_off()
    t=ax.text(0.05, 0.95, text, transform=ax.transAxes, fontsize=14, 
              fontname='Corier', family='monospace', verticalalignment='top', bbox=props)
    return fig



def create_pdf_report(filename='covid_report_D'):
    #
    # create a pdf report
    #
    pdfreport = ende.strftime('%y%m%d') + '_' + filename + '.pdf'
    pp = PdfPages(pdfreport)

    top5_inzidenzen = dinz_dt.sort_values(by='INZ').iloc[-5:]
#    fig1a = print_tabular_overview(top5_inzidenzen, title='Kreise Top 5 der Inzidenzen',)
#    pp.savefig(fig1a)

    top5_inzidenzaenderungen = dinz_dt.sort_values(by='DINZ7').iloc[-5:]
#    fig1b = print_tabular_overview(top5_inzidenzaenderungen, title='Top 5 der Inzidenzänderungen zur Vorwoche',)
#    pp.savefig(fig1b)

    fig1 = text_via_figure(inz_overview(overview))
    pp.savefig(fig1)

    fig2 = didt_hist(dinz_dt.DINZ7.iloc[0:411],ref=' ggüb. vor einer Woche')
    pp.savefig(fig2)

    ref=['Deutschland']
    sample = ['Bayern', 'SK München']
    fig3 = inzidenz_lineplot(ref,sample,filename='inzplot_Bayern')
    pp.savefig(fig3)

    ref=['Deutschland']
    top5_delta_inz = top5_inzidenzaenderungen.index
    fig4 = inzidenz_lineplot(ref,top5_delta_inz,filename='inzplot_top5')
    pp.savefig(fig4)

    fig5 = geoplot(tmpkreise, color='INZ', 
                   cmap='GnBu', bins=[5, 20, 35, 50, 100, 200, 300, 1000],
                   filename='Inzidenzkarte')
    pp.savefig(fig5)

    fig6 = geoplot(tmpkreise, color='DINZ7', cmap='tab10', bins=[0, 100, 200, 300], 
                   title='Delta-Inzidenzen (zu vor 7 Tagen)',filename='Delta-Inzidenz')
    pp.savefig(fig6)

    pp.close()
    return(pdfreport)



In [ ]:
#
# Some constants and names
#
CM = 1/2.54 # cm vs. inch

DEBUG = True

metadata  = (411, 16, 83155031) # Just for checking
pan_begin = pd.Timestamp(2020,1,2)

#Do not forget to reference the data source:
copyright2 = "Datenquellenvermerk: Robert Koch-Institut (RKI), dl-de/by-2-0"
overview = ['Deutschland', 'Bayern', 'SK München']
today = datetime.today().strftime('%y%m%d')


#Filenames for external Data files
covid_file  = "data/" + today + "_RKI_COVID19.csv"
bula_file   = 'landkreise/bundeslaender.shp'
kreise_file = 'landkreise/landkreise.shp'


# Data is stored in subdirectories 'data' and 'landkreise'
data_paths = ['data', 'landkreise']
for i in data_paths:
    if not os.path.exists(i):
        os.makedirs(i)


# Load the covid database via pd.read_csv()

covid  = read_covid_database(covid_file)
bland  = get_bundeslad_geodata(bula_file)
kreise = get_kreise_geodata(kreise_file)
einwohner = extract_einwohnerzahlen(kreise)

start=covid['Meldedatum'].min()
ende=covid['Meldedatum'].max()

check_panbegin = (start == pan_begin)

landkreise=covid['Landkreis'].unique()
bundeslaender=covid['Bundesland'].unique()
check_meta = (len(landkreise), len(bundeslaender), einwohner['Deutschland']) == metadata

result = get_tagesinzidenzen(covid, start, ende)
missing_lk = result.iloc[-1].isna().sum()

neuinfektionen = new_infections(['Deutschland'] + list(bundeslaender) + list(landkreise))



# Now we do some calculations with our daily incidence dataframe

# Calculate Rolling 7 day average and 7 day accumulation    
    
result.fillna(value=0, inplace=True)
result_m7 = result.rolling(7).mean()
result_s7 = result.rolling(7).sum()

# Calculate 7 day Inzidenz per 100.000 inhabitants:

for i in result_s7.columns:
    result_s7[i] = result_s7[i]*100000/ float(einwohner.loc[i])

# Zeitliche Änderung der Inzidenzen berechnen:

dinz_dt = pd.DataFrame(
    {'INZ': result_s7.iloc[-1], 
     'DINZ1': (result_s7.iloc[-1] - result_s7.iloc[-2])/result_s7.iloc[-2]*100,
     'DINZ7': (result_s7.iloc[-1] - result_s7.iloc[-8])/result_s7.iloc[-8]*100, 
     'MELD' : neuinfektionen}
)

# Finally a copy of kreise including  Inzidenzen and Delta-Inzidenzen 
# for plotting the colored geoplots

tmpkreise = kreise.copy()
tmpdinzdt = dinz_dt.reset_index()

tmpdinzdt.rename(columns={'index':'county'}, inplace=True)
tmpkreise=pd.merge(tmpkreise, tmpdinzdt, on='county', how='left')



if DEBUG:
    print(covid_db_stats())

In [ ]:
#
# print data overview
#
print(inz_overview(overview))

In [ ]:
# Get the report!

create_pdf_report()

In [ ]:
# End of main code

In [ ]:
#
# Appendix: Example commands to plot figures and tables
#

In [ ]:
#
# Print overview for a single bundesland
#
fig=plot_landesinfo('Bayern')

In [ ]:
#
# Print table with the top 5 Landkreise with the highest change in Inzidenz
#
top5_inzidenzaenderungen = dinz_dt.sort_values(by='DINZ7').iloc[-5:]
fig=print_tabular_overview(top5_inzidenzaenderungen)

In [ ]:
#
# Just plot some Landkreise or bundeslaender 
# compared to Germany as reference
#
ref=['Deutschland']
sample = ['Bayern', 'SK München']
fig=inzidenz_lineplot(ref,sample,filename='inzplot_Bayern')

In [ ]:
#
# Plot the top 5 Landkreise with the highest change in Inzidenz
# compared to Germany as reference
#
ref=['Deutschland']
top5_dinz_dt7 = dinz_dt.sort_values(by='DINZ7').iloc[-5:]
top5_kreise_delta_inz7 = top5_dinz_dt7.index
fig=inzidenz_lineplot(ref,top5_kreise_delta_inz7,filename='inzplot_top5')

In [ ]:
# Geoplot of all Landkreise and Inzidenz of last available day
fig = geoplot(tmpkreise, color='INZ', cmap='GnBu', 
              bins=[5, 20, 35, 50, 100, 200, 300, 1000],
              filename='Inzidenzkarte')

In [ ]:
# Geoplot of all Landkreise showing the change of Inzidenz compared to 7 days before
fig = geoplot(tmpkreise, color='DINZ7', cmap='tab10', 
              bins=[0, 100, 200, 300], title='Delta-Inzidenzen (zu vor 7 Tagen)',
              filename='Delta-Inzidenz')

In [ ]:
# Histogram showing number of landkreise over Inzidenz change compared to day before last
# available data
fig = didt_hist(dinz_dt.DINZ1.iloc[0:411],ref=' ggüb. vor einem Tag', file='Histogramm1')

In [ ]:
# Histogram showing number of landkreise over Inzidenz change compared to week before
#
fig = didt_hist(dinz_dt.DINZ7.iloc[0:411],ref=' ggüb. vor einer Woche', file='Histogramm7')

In [ ]:
a=text_via_figure(inz_overview(overview))

In [ ]:
#
# Generate overview of all Bundesländer
#

imgs=[]
for i in bundeslaender:
    plot_landesinfo(i, savefig=True)
    imgs.append(ende.strftime('%Y%m%d') + "_" + 'land_info' + '_' + i + '.png')

fig, axs = plt.subplots(8, 2, figsize=(20, 56))

axs = axs.flatten()
for img, ax in zip(imgs, axs):
    image = mpimg.imread(img)
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.imshow(image)

fig.subplots_adjust(hspace = 0)

In [ ]:
### Debug and test section

In [ ]:
#
# Just some lines to test and look into variables
#